In [1]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
train2 =  pd.read_csv('../data/minseok_EDA2_train.csv')
test2 = pd.read_csv('../data/minseok_EDA2_test.csv')

In [3]:
train = pd.read_csv('../data/real_final_train.csv')
test = pd.read_csv('../data/real_final_test.csv')

In [4]:
train.shape

(15000, 20)

In [5]:
train2.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,hand,religion,orientation,voted,married,familysize,ASD,nerdiness,Qs_Mach,age_cat
0,0.219430,0.684909,0.645424,0.689224,0.270300,0.580272,0.649525,0.664648,0.287443,0.462615,...,0.550968,0.567721,0.617225,0.549084,0.560104,0.547405,0.54676,1,3.653846,0.559000
1,0.549147,0.490676,0.505430,0.576165,0.538382,0.754382,0.494185,0.537294,0.460687,0.462615,...,0.552290,0.597924,0.521066,0.562683,0.515796,0.547405,0.54676,1,3.269231,0.523114
2,0.549147,0.684909,0.645424,0.576165,0.490014,0.754382,0.649525,0.664648,0.541905,0.523798,...,0.552290,0.597924,0.600253,0.549084,0.529865,0.547405,0.54676,1,3.692308,0.523114
3,0.549147,0.490676,0.505430,0.382335,0.538382,0.427424,0.435168,0.664648,0.460687,0.523798,...,0.550968,0.565894,0.521066,0.549084,0.560104,0.565604,0.54676,1,3.500000,0.564478
4,0.549147,0.490676,0.505430,0.576165,0.490014,0.427424,0.494185,0.413060,0.460687,0.523798,...,0.550968,0.567721,0.521066,0.549084,0.560104,0.573536,0.54676,0,3.461538,0.564478


In [6]:
train.drop(columns=['hand.1', 'religion.1'], axis=1, inplace=True)
test.drop(columns=['hand.1', 'religion.1'], axis=1, inplace=True)

In [7]:
train2.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'introelapse', 'testelapse',
       'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6',
       'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2', 'VCL3', 'VCL4',
       'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10', 'VCL11', 'VCL12',
       'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education', 'urban', 'gender',
       'engnat', 'hand', 'religion', 'orientation', 'voted', 'married',
       'familysize', 'ASD', 'nerdiness', 'Qs_Mach', 'age_cat'],
      dtype='object')

In [8]:
train2[train2.columns[55:]]

,education,urban,gender,engnat,hand,religion,orientation,voted,married,familysize,ASD,nerdiness,Qs_Mach,age_cat
0,0.543273,0.565089,0.652949,0.573883,0.550968,0.567721,0.617225,0.549084,0.560104,0.547405,0.546760,1,3.653846,0.559000
1,0.533187,0.571283,0.550584,0.573883,0.552290,0.597924,0.521066,0.562683,0.515796,0.547405,0.546760,1,3.269231,0.523114
2,0.543273,0.565089,0.545191,0.515462,0.552290,0.597924,0.600253,0.549084,0.529865,0.547405,0.546760,1,3.692308,0.523114
3,0.591922,0.525160,0.545191,0.573883,0.550968,0.565894,0.521066,0.549084,0.560104,0.565604,0.546760,1,3.500000,0.564478
4,0.591922,0.571283,0.550584,0.515462,0.550968,0.567721,0.521066,0.549084,0.560104,0.573536,0.546760,0,3.461538,0.564478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.543273,0.571283,0.550584,0.573883,0.552290,0.565894,0.578521,0.549084,0.560104,0.544640,0.546760,0,3.346154,0.564478
14996,0.533187,0.565089,0.550584,0.515462,0.552290,0.495370,0.521066,0.562683,0.515796,0.544640,0.546760,1,4.038462,0.523114
14997,0.543273,0.571283,0.550584,0.573883,0.552290,0.565894,0.600253,0.562683,0.560104,0.544640,0.664114,1,4.615385,0.559000
14998,0.550553,0.571283,0.550584,0.573883,0.552290,0.567721,0.617225,0.549084,0.515796,0.565604,0.664114,0,4.115385,0.559000


In [9]:
one = train2[train2.columns[0:26]]
one_test = test2[test2.columns[0:26]]

In [10]:
two = train2[train2.columns[39:55]]
two_test = test2[test2.columns[39:55]]

In [11]:
two['VCL_cal']  = two.sum(axis=1) / 16
two_test['VCL_cal']  = two_test.sum(axis=1) / 16

C:\Users\Home\AppData\Local\Temp/ipykernel_23580/1563701938.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two['VCL_cal']  = two.sum(axis=1) / 16
C:\Users\Home\AppData\Local\Temp/ipykernel_23580/1563701938.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_test['VCL_cal']  = two_test.sum(axis=1) / 16


In [12]:
two

,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,VCL_cal
0,0.557163,0.564123,0.496106,0.557552,0.559536,0.547971,0.544852,0.526920,0.551315,0.556307,0.536068,0.536299,0.456042,0.567572,0.557508,0.55517,0.541906
1,0.557163,0.564123,0.604637,0.557552,0.559536,0.547971,0.581638,0.526920,0.551315,0.556307,0.536068,0.536299,0.584129,0.567572,0.557508,0.55517,0.558994
2,0.557163,0.564123,0.496106,0.557552,0.559536,0.547971,0.581638,0.587876,0.551315,0.556307,0.536068,0.536299,0.584129,0.567572,0.557508,0.55517,0.556021
3,0.557163,0.564123,0.496106,0.557552,0.559536,0.547971,0.544852,0.526920,0.551315,0.556307,0.536068,0.536299,0.584129,0.567572,0.557508,0.55517,0.549912
4,0.557163,0.564123,0.496106,0.557552,0.559536,0.547971,0.544852,0.587876,0.551315,0.556307,0.536068,0.536299,0.456042,0.567572,0.458472,0.55517,0.539526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.557163,0.564123,0.604637,0.557552,0.559536,0.547971,0.544852,0.526920,0.551315,0.556307,0.536068,0.618263,0.584129,0.567572,0.557508,0.55517,0.561818
14996,0.557163,0.564123,0.496106,0.557552,0.559536,0.547971,0.544852,0.526920,0.551315,0.556307,0.536068,0.536299,0.456042,0.567572,0.557508,0.55517,0.541906
14997,0.557163,0.564123,0.496106,0.557552,0.559536,0.547971,0.544852,0.526920,0.551315,0.556307,0.536068,0.536299,0.584129,0.567572,0.557508,0.55517,0.549912
14998,0.557163,0.564123,0.604637,0.557552,0.559536,0.547971,0.544852,0.587876,0.551315,0.556307,0.536068,0.618263,0.584129,0.567572,0.557508,0.55517,0.565627


In [27]:
drop_train = pd.concat([train2['TIPI1'], train2['TIPI6'], train2['TIPI7'], train2['TIPI8'], train2['TIPI10'],
          one, train2['education'], train2['urban'], train2['religion'], train2['orientation'], train2['age_cat'], train2['ASD'], 
                        two , train2['nerdiness']
          ], axis=1)

In [28]:
drop_test = pd.concat([test2['TIPI1'], test2['TIPI6'], test2['TIPI7'], test2['TIPI8'], test2['TIPI10'],
          one_test, test2['education'], test2['urban'], test2['religion'], test2['orientation'], test2['age_cat'], test2['ASD'],
                       two_test
          ], axis=1)

In [29]:
drop_train.shape

(15000, 55)

In [30]:
drop_test.shape

(35452, 54)

In [31]:
# one = train[train.columns[0:2]]
# one_test = test[test.columns[0:2]]

In [32]:
# two = train[train.columns[14]]
# two_test = test[test.columns[13]]

In [33]:
# final_train = pd.concat([train2, two], axis=1)
# final_test = pd.concat([test2, two_test], axis=1)

In [34]:
# final_train.shape

In [35]:
# final_test.shape

In [36]:
train,test = drop_train, drop_test

In [37]:
columns = [col for col in train.columns.to_list() if col not in ['nerdiness']]

In [38]:
data = train[columns]
target = train['nerdiness']

In [39]:
def objective(trial, data=data, target=target):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2, random_state=42)
    param = {
        'tree_method': 'gpu_hist',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree' : trial.suggest_categorical('colsample_bytree', [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample' : trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-2, 0.2),
        'n_estimators' : trial.suggest_categorical('n_estimators', [3000, 5000, 7000, 10000]),
        'max_depth' : trial.suggest_categorical('max_depth', [3, 5, 6, 7, 8, 9, 10, 12]),
        'random_state' : trial.suggest_categorical('random_state', [42, 777, 20171184]),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'use_label_encoder' : False,
        'eval_metric': 'auc'
    }
    model = xgb.XGBClassifier(**param)
    
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100, verbose=False)
    
    preds = model.predict(test_x)
    
    auc = roc_auc_score(test_y, preds)
    
    return auc

In [40]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-08-08 18:59:05,958] A new study created in memory with name: no-name-8e31227a-b93a-448b-a086-5b45dbc95900
[I 2022-08-08 18:59:08,090] Trial 0 finished with value: 0.7073019701801817 and parameters: {'lambda': 0.17743042263707165, 'alpha': 0.44469153437316883, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.12805240332657933, 'n_estimators': 10000, 'max_depth': 7, 'random_state': 42, 'min_child_weight': 238}. Best is trial 0 with value: 0.7073019701801817.
[I 2022-08-08 18:59:09,687] Trial 1 finished with value: 0.7145855930236951 and parameters: {'lambda': 0.011036862301016774, 'alpha': 0.001728493441548897, 'colsample_bytree': 0.9, 'subsample': 0.4, 'learning_rate': 0.1695510647028013, 'n_estimators': 7000, 'max_depth': 5, 'random_state': 777, 'min_child_weight': 110}. Best is trial 1 with value: 0.7145855930236951.
[I 2022-08-08 18:59:11,504] Trial 2 finished with value: 0.7118240490413518 and parameters: {'lambda': 0.002542621012570366, 'alpha': 0.0070495320114

[I 2022-08-08 19:03:24,068] Trial 22 finished with value: 0.7455224404116279 and parameters: {'lambda': 0.003908555349793681, 'alpha': 0.003528051639622016, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.01513342538329646, 'n_estimators': 3000, 'max_depth': 9, 'random_state': 777, 'min_child_weight': 33}. Best is trial 16 with value: 0.7757941968453373.
[I 2022-08-08 19:03:37,536] Trial 23 finished with value: 0.7733969014586133 and parameters: {'lambda': 0.011841557013137115, 'alpha': 0.16897894292861634, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.047464716332456684, 'n_estimators': 5000, 'max_depth': 9, 'random_state': 777, 'min_child_weight': 5}. Best is trial 16 with value: 0.7757941968453373.
[I 2022-08-08 19:03:41,371] Trial 24 finished with value: 0.7215872604728217 and parameters: {'lambda': 0.01752915984434817, 'alpha': 0.24005240923272997, 'colsample_bytree': 0.4, 'subsample': 1.0, 'learning_rate': 0.05261829748346171, 'n_estimators': 5000, '

Number of finished trials: 30
Best trial: {'lambda': 0.0045556616137676035, 'alpha': 0.02069897470317937, 'colsample_bytree': 1.0, 'subsample': 1.0, 'learning_rate': 0.02984206538600726, 'n_estimators': 3000, 'max_depth': 9, 'random_state': 777, 'min_child_weight': 7}


In [41]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_n_estimators,params_random_state,params_subsample,state
0,0,0.707302,2022-08-08 18:59:05.959020,2022-08-08 18:59:08.089028,0 days 00:00:02.130008,0.444692,0.6,0.177430,0.128052,7,238,10000,42,0.6,COMPLETE
1,1,0.714586,2022-08-08 18:59:08.090029,2022-08-08 18:59:09.687266,0 days 00:00:01.597237,0.001728,0.9,0.011037,0.169551,5,110,7000,777,0.4,COMPLETE
2,2,0.711824,2022-08-08 18:59:09.688267,2022-08-08 18:59:11.504675,0 days 00:00:01.816408,0.007050,0.9,0.002543,0.091208,6,147,10000,777,0.7,COMPLETE
3,3,0.719735,2022-08-08 18:59:11.505676,2022-08-08 18:59:14.868744,0 days 00:00:03.363068,0.072075,0.5,0.023667,0.048509,10,192,10000,777,1.0,COMPLETE
4,4,0.712689,2022-08-08 18:59:14.869745,2022-08-08 18:59:15.940488,0 days 00:00:01.070743,0.737619,0.6,0.050954,0.152605,12,173,7000,777,0.9,COMPLETE
5,5,0.704138,2022-08-08 18:59:15.941488,2022-08-08 18:59:17.101748,0 days 00:00:01.160260,1.814879,0.7,0.502786,0.093265,6,280,5000,777,0.4,COMPLETE
6,6,0.717958,2022-08-08 18:59:17.102749,2022-08-08 18:59:20.112602,0 days 00:00:03.009853,0.008702,0.4,0.660154,0.092844,3,69,10000,42,0.7,COMPLETE
7,7,0.713373,2022-08-08 18:59:20.113602,2022-08-08 18:59:20.758748,0 days 00:00:00.645146,0.001397,0.8,1.910322,0.195668,8,146,7000,20171184,0.6,COMPLETE
8,8,0.707147,2022-08-08 18:59:20.759749,2022-08-08 18:59:23.555147,0 days 00:00:02.795398,0.004444,0.6,0.001563,0.023187,7,139,10000,42,0.4,COMPLETE
9,9,0.711960,2022-08-08 18:59:23.556147,2022-08-08 18:59:25.381160,0 days 00:00:01.825013,0.001214,0.7,0.001418,0.069463,7,227,10000,20171184,0.7,COMPLETE


In [42]:
Best_trial = study.best_trial.params
Best_trial["tree_method"] = 'gpu_hist'
Best_trial

{'lambda': 0.0045556616137676035,
 'alpha': 0.02069897470317937,
 'colsample_bytree': 1.0,
 'subsample': 1.0,
 'learning_rate': 0.02984206538600726,
 'n_estimators': 3000,
 'max_depth': 9,
 'random_state': 777,
 'min_child_weight': 7,
 'tree_method': 'gpu_hist'}

In [43]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=3, random_state=42, shuffle=True)
auc=[]
n=0
for trn_idx, test_idx in kf.split(train[columns], train['nerdiness']):
    X_tr, X_val = train[columns].iloc[trn_idx], train[columns].iloc[test_idx]
    y_tr, y_val = train['nerdiness'].iloc[trn_idx], train['nerdiness'].iloc[test_idx]
    model = xgb.XGBClassifier(**Best_trial)
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
    preds += model.predict(test[columns])/kf.n_splits
    auc.append(roc_auc_score(y_val, model.predict(X_val)))
    print(f'fold: {n+1} ==> auc: {auc[n]}')
    n += 1

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:05:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold: 1 ==> auc: 0.7647010768081663
[19:05:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


fold: 2 ==> auc: 0.7612065185659518
[19:05:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


fold: 3 ==> auc: 0.7586180504928122


In [44]:
np.mean(auc)

0.76150854862231

In [ ]:
preds

In [ ]:
len(preds)

In [ ]:
sub = pd.read_csv('../data/sample_submission.csv')

In [ ]:
sub.head()

In [ ]:
sub['nerdiness']=preds
sub.to_csv('../submission/xgboost_optuna.csv, index=False)

In [ ]:
sub.shape